In [23]:
import numpy as np
import pandas as pd 
from scipy.linalg import expm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from netneurotools.metrics import distance_wei_floyd

In [24]:
sc_mats = np.load('/home/shir/Documents/connectomics/coupling/216/data/hc_fbc_mats.npy') #load a 3D numpyarray with structural matrices (2D) for each control participant
fc_mats = np.load('/home/shir/Documents/connectomics/coupling/216/data/hc_fc_mats.npy')
struc = sc_mats[:,:,100]
func = fc_mats[:,:,100]
leb = pd.read_csv('/home/shir/Documents/connectomics/coupling/216/atlas_labels.csv',header=None)

In [25]:
def short_path(mat):
    """
    This function takes an adjacency matrix and returns a shortest path length matrix between each pair of nodes.
    :param mat: a matrix of the connection strength between each pair of nodes
    :return: shortest path matrix
    """
    # Create a copy of the matrix to avoid modifying the original
    mati = np.array(mat, copy=True)
    # Replace zero entries with np.inf
    mati[mati == 0] = 1e-10
    # Compute the inverse of the matrix for weights (path length in terms of cost)
    rec_mat = 1 / mati
    np.fill_diagonal(rec_mat, 0)
    # Compute shortest paths using Floyd-Warshall algorithm
    short, pre = distance_wei_floyd(rec_mat)
    return short   
    
def communicability(adjmat, normalize=True):
    """
    This function will take an adjacency matrix and return a communicability matrix
    the communicability matrix is the weighted  sum  of  all  paths  and  walks  between all pairs of nodes
    :param adjmat: a matrix of the connection strength between each pair of nodes
    :param normalize: default to normalize values
    :return: communicability matrix
    """
    adjmati = np.array(adjmat, dtype=float) 
    if normalize:
        norm = np.linalg.eigvals(adjmati).max()
        adjmati = adjmati / norm

    return expm(adjmati)

In [26]:
shorti=short_path(struc)
commi=communicability(struc)

In [27]:
def get_predictor_vectors(mats, node, scope):
    """this function takes a list of matrices and a number and return for each matrix a 1D coulmn with the values for that number index.
    depands of the scope, the vector will contain whole brain connections (to all regions), within network connections (to regions in the same network),
    between network connections (to all regions that are not in the same network), or cortical connections (only to cortical regions). 
    Important! labels here match the schafer and tian stlases, adjust if needed.
    #there's probably a cleaner way to do this#
    :param mats: list of structural matrices (same shape)
    :param node: number to use as index
    :param scope: 'between', 'within', 'whole', 'cortical'
    :return: a vector for the node number (excluding self connection)
    """
    selected = leb.iloc[node,0] #get the label of the region (the node)
    nets = ["Vis", "SomMot", "DorsAttn", "SalVentAttn", "Limbic", "Cont", "Default", "-"] #for subcortex I am using "-" as it's not in the schaefer labels and only apears in all the Tian
    if scope == 'between':
        net_indices = [node] #node number as list to be used index and later add to it more indices
    else:
        net_indices = []
    
    for net in nets: #loop over the labels of the networks
        if net in selected: 
            for i in range(len(leb)): 
                check = leb.iloc[i,0] ###if the network in the loop matched the label of the node, loop over the number of regions, and 
                #pull the region label one by one
                if net in check and i != node: 
                    net_indices.append(i) ##if that network is also in the label of the region being checked, 
                    #and it's not the same region (self connection), add that region (its index) to the list of indices to be used for analysis scope
    net_indices = np.unique(net_indices) #avoid duplicates
    print(net_indices)
    result = [] #store the vectors
    for mat in mats: #loop over the matrices
        column_vector = mat[:, node] #extract the column of the region by node number as index
        if scope == "between":
            column_vector = np.delete(column_vector, net_indices) #if between network, remove from the vector the indices of regions in the same network
            result.append(column_vector) #add the list of vectors (for all mats)
        elif scope == "within":
            column_vector = column_vector[net_indices] #if within network, keep only indices of regions within that network
            result.append(column_vector)
        elif scope == "whole":
            column_vector = np.delete(column_vector, node) #if whole brain, use all regions, delete self connection
            result.append(column_vector)
        elif scope == "cortical":
            column_vector = column_vector[:200] #if cortical, use indices until 200 as these are the Schaefer one
            if node < 200:
                column_vector = np.delete(column_vector, node) #delete self connections if the node is in cortical range (discard later anyway if it's over 200)
            result.append(column_vector)
        else:
            print("scope does not match") 
    return result

In [33]:
ss = get_predictor_vectors([struc, shorti, commi], 5, 'whole')

[  0   1   2   3   4   6   7   8   9  10  11  12  13 100 101 102 103 104
 105 106 107 108 109 110 111 112 113 114]


In [34]:
def get_predicted_vector(mat, node, scope):
    """this function takes a matrix and a number and return a 1D coulmn with the values for that number index
    :param mat: a functional matrix
    :param node: number to use as index
    :param scope: 'between', 'within', 'whole', 'cortical'
    :return: a vector for the node number (excluding self connection)
    """
    selected = leb.iloc[node,0]
    nets = ["Vis", "SomMot", "DorsAttn", "SalVentAttn", "Limbic", "Cont", "Default", "-"] #for subcortex I am using "-" as it's not in the schaefer labels and only apears in all the Tian
    
    if scope == 'between':
        net_indices = [node] #node number as list to be used index and later add to it more indices, if between, add seld connection to be removed
    else:
        net_indices = []    

    for net in nets:
        if net in selected:
            for i in range(len(leb)):
                check = leb.iloc[i,0]
                if net in check and i != node:
                    net_indices.append(i)
    net_indices = np.unique(net_indices)
    
    column_vector = mat[:, node]
    if scope == "between":
        column_vector = np.delete(column_vector, net_indices)
    elif scope == "within":
        column_vector = column_vector[net_indices]
    elif scope == "whole":
        column_vector = np.delete(column_vector, node)
    elif scope == "cortical":
        column_vector = column_vector[:200]
        if node < 200:
            column_vector = np.delete(column_vector, node)
    else:
        print("scope does not match") 
    return column_vector



In [35]:
ff=get_predicted_vector(func,5,'whole')

In [42]:
def predict_function(predictors, functional):
    """this function takes a list of structural connectivity vectors and a functional connectivity vector and performs a linrear regression to prdict functional from strctural.
    it returns the squered R between the real functional vector and the predicted functional vector.
    :param predictors: list of structural connectivity vectors
    :param functiobal: functional connectivity vector
    :return: a predicted functional matrix"""

    model = LinearRegression() 
    predictors_arr = np.transpose(np.array(predictors)) #dimentions to fit
    print(predictors_arr.shape)
    scaler = StandardScaler()
    predictors_scaled = scaler.fit_transform(predictors_arr) #standarise the predictors
    model.fit(predictors_scaled, functional) #use linear regression model to predict functional from structural
    r_squared = model.score(predictors_scaled, functional) #get the R2 between observed and predicted   
    N = len(functional)
    print('N',N)
    p = predictors_scaled.shape[1]
    print('p',p)
    adjusted_r_squared = 1 - ((1 - r_squared) * (N - 1) / (N - p - 1)) #compute the adjusted R2
    return adjusted_r_squared

In [43]:
predict_function(ss, ff)

(215, 3)
N 215
p 3


0.1862753908111704